In [8]:
import numpy as np
import matplotlib.pyplot as plt

# Load Data
Load the cleaned and filtered data

In [9]:
all_data = np.load('processed_data/tune_data.npy', allow_pickle=True)[:, 1:, :]
labels = np.load('processed_data/tune_labels.npy', allow_pickle=True)
labels = [i-1 for i in labels]
sr = 1000

# Finding Onset
We are applying moving average window based onset detection:

1. Root Mean Square (RMS) Calculation: The `get_rms`` function computes the RMS value of the signal data over specified frame lengths and steps. RMS is a statistical measure of the magnitude of a varying quantity and is commonly used to determine the loudness of a signal.

2. Onset Detection: The `find_onset_index` function determines the onset point by identifying the first instance where the RMS of the signal exceeds a specified threshold. This threshold is a crucial parameter in detecting the precise moment when the onset happens.

3. Onset Time Calculation: The `find_onset_time` function integrates RMS computation and onset detection to return the time index at which the onset occurs in the signal.

4. Signal Smoothing: The `moving_average` function smooths the RMS values using a moving average, which helps in reducing noise and making the onset detection more accurate.

5. Normalization: The `normalize` function scales the RMS values to a normalized range, ensuring consistency in onset detection across different signals.

6. Comprehensive Onset Detection: The `get_onset` function combines RMS calculation, smoothing, normalization, and threshold-based onset detection to provide a robust method for determining the onset of a gesture in our sEMG signal.

7. Average Onset Calculation: The `average_onset` function computes the average onset time over multiple channels of the signal, enhancing the reliability of the onset detection.

8. Data Segment Retrieval: Finally, the `get_onset_data` function retrieves a specific segment of the data (0.6 seconds) starting from the detected onset point, useful for further analysis or processing of the signal starting at its onset.

In [10]:
def get_rms(data, frame_length, step):
    rms = []
    for i in range(0, len(data)-frame_length+1, step):
        rms.append(np.sqrt(np.mean(data[i:i+frame_length]**2)))
    return np.array(rms).squeeze()

def find_onset_index(rms):
    threshold = 0.4
    for i in range(len(rms)):
        if rms[i] > threshold:
            return i
        
def find_onset_time(data, threshold, frame_length, step):
    rms = get_rms(data, frame_length, step)
    index = find_onset_index(rms)
    return index * step

def moving_average(rms):
    w = 1
    return np.convolve(rms, np.ones(w), 'valid') / w

def normalize(rms):
    #min max scaler
    return np.array([n/(max(rms)-min(rms)) for n in rms])

def get_onset(signal, frame_size, step, threshold=0.4):
    rms = get_rms(signal, frame_size, step)
    rms = moving_average(rms)
    rms = normalize(rms)
    index = np.where(rms >threshold)[0][0]
    onset = index * step
    return onset

def average_onset(signal, frame_size, step, threshold=0.5):
    onsets = 0
    for i in range(4):
        onsets += get_onset(signal[i], frame_size, step)
    return int(onsets/4)

def get_onset_data(data):
    onset = average_onset(data, 300, 100)
    onset_data = []
    if onset > 1400-600:
        onset = 800
    return data[:, onset:onset+600]


In [11]:
all_onset_data = []
for i in range(len(all_data)):
    all_onset_data.append(get_onset_data(all_data[i]))
all_onset_data = np.array(all_onset_data)

In [12]:
np.save('processed_data/tune_onset_data.npy', all_onset_data)